# 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B
$$
$$
h_t = tanh(a_t)
$$

In [92]:
import numpy as np

In [139]:
def forward(x):
    h = np.zeros((batch_size, n_nodes))
    for n in range(n_sequences):
        h = np.tanh(x[:, n, :] @ w_x + h @ w_h + b)
    return h

# 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [141]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

```python
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)
```

In [142]:
forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

# ※ 問題３未完成です。

# 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

In [40]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
        
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        W :
        """
        W = self.sigma * np.random.randn(n_nodes1, n_nodes2)
        return W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数

        Returns
        ----------
        B :
        """
        B = self.sigma * np.random.randn(1, n_nodes2)
        return B

In [41]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        layer.Wx -= self.lr*layer.dWx
        layer.Wh -= self.lr*layer.dWh
        layer.B -= self.lr*layer.dB
        return layer

In [42]:
class Tanh:
    '''
    ハイパボリックタンジェント関数
    '''
    def forward(self, A):
        """
        フォワード
        Parameters
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        ----------
        Z : 次の形のndarray, shape (batch_size, n_nodes1)
            出力
        """ 
        self.A = A
        Z = np.tanh(self.A)
        return Z
    
    def backward(self, dZ):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dA = dZ * (1 - np.tanh(self.A)**2)
        return dA

In [89]:
class SimpleRNN:
    
    def __init__(self,):

        self.Wx = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
        self.Wh = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
        self.b = np.array([1, 1, 1, 1]) # (n_nodes,)
        self.dB = 0
        self.dWx = 0
        self.dWh = 0
        self.activator = Tanh()
        self.optimizer = SGD()
        self.h_his = []
    
    def forward(self, x):
        self.x = x.copy()
        batch_size = x.shape[0] 
        self.n_sequences = x.shape[1] 
        n_features = x.shape[2] 
        n_nodes = w_x.shape[1] 
        h = np.zeros((batch_size, n_nodes))
        for n in range(self.n_sequences):
            A = x[:, n, :] @ self.Wx + h @ self.Wh + self.b
            h = self.activator.forward(A)
            self.h_his.append(h)
        return h
    
    def backward(self, dh):
        dA = self.activator.backward(dh)
        for n in range(self.n_features)[::-1]:
            self.dB += np.sum(dA, axis=0)
            self.dWx += x[:, n, :].T @ dA
            self.dWh += h[n].T @ dA

        dx = dA @ Wx
        dh = dA @ Wh
        return dx, dh

In [90]:
rnn = SimpleRNN()
rnn.forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])